In [1]:
import pypsa, numpy as np, pandas as pd, matplotlib.pyplot as plt
from build_network import build_and_optimize_network
from build_network import extract_detailed_results


In [4]:
#Load the evolve dataset and set its date time index
timeseries_data_full = pd.read_csv('Inputs_GB_LTW_2030_baseline.csv')
start_date = '2030-01-01 00:00:00'
datetime_index = pd.date_range(start=start_date, periods=len(timeseries_data_full), freq='h')

# Set the new DatetimeIndex and remove the old 'Period' column
timeseries_data_full.index = datetime_index
timeseries_data_full = timeseries_data_full.drop(columns=['Period'])
timeseries_data_full.head(5)

,Demand,PV_Z1,PV_Z2,PV_Z3,PV_Z4,PV_Z5,PV_Z6,PV_Z7,PV_Z8,PV_Z9,...,Z3_wave,Z8_wave,Z9_wave,Z1_tidal,Z3_tidal,Z5_tidal,Z7_tidal,Z9_tidal,DSR_min,DSR_max
2030-01-01 00:00:00,26276.86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.847830,0.572871,0.537692,0.207735,0.009276,0.025925,0.011799,0.019891,-1.0,1.0
2030-01-01 01:00:00,25831.43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.847830,0.572546,0.533920,0.035931,0.108594,0.176336,0.195456,0.212510,-1.0,1.0
2030-01-01 02:00:00,24952.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.845383,0.567793,0.527585,0.205646,0.395722,0.457483,0.598944,0.554929,-1.0,1.0
2030-01-01 03:00:00,24178.14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.837562,0.558284,0.522157,0.669624,0.522340,0.564950,0.749965,0.594264,-1.0,1.0
2030-01-01 04:00:00,23628.61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.827409,0.548857,0.516379,0.787130,0.407776,0.506456,0.539666,0.405528,-1.0,1.0


In [5]:
# Subsample the data to get one entry per day.
# if you want to sample every 25th hour, you can use:
# indices = np.arange(0, len(timeseries_data_full), 25)
# use check time.ipynb to see how long the model takes to run with different subsampling.          
indices = np.linspace(0, len(timeseries_data_full)-1, 365, dtype=int)
subsampled_data = timeseries_data_full.iloc[indices]
len (subsampled_data)

365

In [11]:

# --- Data Dictionaries for Renewables + Storage Scenario ---

# add carriers that are relevant to the model
relevant_carriers = [
    "onwind", "offwind", "tidal",
    "battery", "hydroelectric_ph", "DSR", "load_shed", "AC"
]

# Base marginal costs for each technology type, in GBP/MWh.
base_marginal_costs = {
    "onwind": 0, "offwind": 0, "tidal": 0,
    "battery": 1, "hydroelectric_ph": 60, "DSR": 30, "load_shed": 10e6
}

# Base capital costs for each technology type, in GBP/MW
base_capital_costs = {
    "onwind": 1200000,
    "offwind": 1810000,
    "tidal": 4933333.3333,
    "battery": 648000,
    "hydroelectric_ph": 2045940,
    "AC": 119000
}

# --- Penalties: scores between 1-10 for operational and 0-1 for construction penalties ---

operational_penalties = {
    "onwind": {
        # A mid-range penalty reflecting the continuous noise and visual impact on local communities and birds.
        "biodiversity & noise": 5,
        # A moderately high penalty due to the large land area turbines occupy, even with some multi-use possible.
        "land use": 6
    },
    "offwind": {
        # A significant penalty due to the operational noise from turbines affecting sensitive marine mammals and ecosystems.
        "biodiversity & noise": 6,
        # A very low penalty as the only land use is for a small onshore substation.
        "land use": 2
    },
    "tidal": {
        # A very high penalty, reflecting the major disruption that operating turbines cause in ecologically concentrated tidal zones.
        "biodiversity & noise": 8,
        # A very low penalty, similar to offshore wind, as the land footprint is minimal.
        "land use": 2
    },
    "battery": {
        # A low penalty, representing the constant but localized hum from cooling systems and inverters.
        "biodiversity & noise": 2,
        # A low penalty due to the small, contained physical footprint of the facility.
        "land use": 2
    },
    "hydroelectric_ph": {
        # A significant penalty for the major and permanent disruption to river ecosystems and fish migration from daily operations.
        "biodiversity & noise": 6,
        # A high penalty reflecting the permanent loss of land from flooding it for two large reservoirs.
        "land use": 7
    },
}
# Each penalty is an independent score between 0 and 1.
# Keys have been renamed for clarity in plots and analysis.
construction_penalties = {
    "onwind": {
        "ghg": 0.15,                  # Low GHG: Based on concrete for foundation and steel for tower. Nudged up to be more competitive with offshore.
        "biodiversity & noise": 0.18, # Low impact: Construction noise is temporary and localized. Nudged up to be closer to offshore.
        "land use": 0.90,             # High impact: Occupies a large area, though the land can often be multi-use (e.g., farming).
        "grid upgrades": 0.60,        # Very High impact: Reflects the high system cost of managing intermittency with storage/backup.
        "reliability": 0.85           # High penalty: Highly variable output dependent on weather.
    },
    "offwind": {
        "ghg": 0.20,                  # Medium GHG: Larger turbines and subsea foundations are material-intensive. Lowered to be more competitive.
        "biodiversity & noise": 0.20, # Medium impact: Pile driving for foundations is very disruptive to marine life.
        "land use": 0.25,             # Low-Medium impact: Requires significant onshore substation and port facilities for construction and maintenance.
        "grid upgrades": 0.70,        # High impact: High cost of subsea cables combined with the need for grid support for intermittency.
        "reliability": 0.65           # Medium-High penalty: More consistent and stronger than onshore, but still variable.
    },
    "tidal": {
        "ghg": 0.40,                  # High GHG: Extremely material-intensive (steel/concrete) due to immature tech and harsh marine environment.
        "biodiversity & noise": 0.45, # High impact: Major construction disruption in sensitive and concentrated tidal ecosystems.
        "land use": 0.10,             # Very low impact: Minimal land use, primarily for an onshore substation and grid connection.
        "grid upgrades": 0.20,        # Very Low impact: Predictability means far less need for costly grid support, making it the best option for grid stability.
        "reliability": 0.10           # Very low penalty: Not constant, but 100% predictable based on tides, making it highly reliable for the grid.
    },
    "battery": {
        "ghg": 0.30,                  # Medium-High GHG: Dominated by energy-intensive mining and processing of raw materials (lithium, cobalt).
        "biodiversity & noise": 0.05, # Very low impact: Standard, temporary construction site noise.
        "land use": 0.15,             # Low impact: A concentrated, small footprint for the facility itself.
        "grid upgrades": 0.10,        # Medium impact: Requires robust substation upgrades to handle high-power, two-way energy flow (charging and discharging).
        "reliability": 0.05           # Very low penalty: A primary solution to intermittency; output is dispatchable and highly reliable.
    },
    "hydroelectric_ph": {
        "ghg": 0.25,                  # Medium GHG: Involves massive amounts of concrete for dam and reservoir construction.
        "biodiversity & noise": 0.30, # High impact: A major civil engineering project with significant, long-term ecosystem disruption.
        "land use": 0.95,             # Very high impact: Involves the permanent and total consumption of land by flooding it for two large reservoirs.
        "grid upgrades": 0.30,        # Medium impact: Requires a very high-capacity connection to the grid to handle large power flows.
        "reliability": 0.05           # Very low penalty: A primary solution to intermittency; dispatchable and very reliable output.
    },
}




In [12]:

# ========================================================================================
# SCRIPT TO RUN A TRADE-OFF ANALYSIS BETWEEN DIFFERENT POLICY PRIORITIES
#
# Methodology:
# 1. A baseline "Economic Only" scenario is run with all penalty weights set to zero. This
#    provides a purely cost-optimized reference point.
# 2. The script then iterates ("sweeps") through each priority defined below.
# 3. For each priority, its importance (weight) is varied according to `weight_range`.
# 4. The remaining weight (out of a total of 100) is distributed *evenly* among all
#    other priorities. This ensures the total weight sum is always constant, allowing for
#    a fair comparison of trade-offs.
# ========================================================================================

# --- 1. Define the Scope of the Analysis ---

# A list of all the non-economic priorities that will be analyzed.
priorities_to_sweep = [
    'ghg',
    'biodiversity & noise',
    'land use',
    'grid upgrades',
    'reliability'
]

# The different levels of importance (weights) that will be assigned to the primary priority in each sweep.
weight_range = [0, 50, 100]

# An empty list to store the results dictionary from each completed scenario.
all_sweep_results = []

# The constant sum that all priority weights must add up to in any given scenario.
TOTAL_WEIGHT_SUM = 100

# --- 2. Run the "Economic Only" Baseline Scenario ---
# This run establishes a purely cost-optimized reference point against which all
# other trade-off scenarios can be compared.
print("\n========================================================")
print("--- Running Economic Baseline (All Penalties Zero) ---")
print("========================================================")

# For the baseline, all penalty weights are explicitly set to zero.
economic_only_weights = {p: 0 for p in priorities_to_sweep}
scenario_name = "Economic_Baseline"

# Use a try-except block to gracefully handle any potential errors during optimization.
try:
    # Build and optimize the network with zero penalty weights.
    network = build_and_optimize_network(
        scenario_name=scenario_name,
        penalty_weights=economic_only_weights,
        subsampled_data=subsampled_data,
        relevant_carriers=relevant_carriers,
        base_marginal_costs=base_marginal_costs,
        base_capital_costs=base_capital_costs,
        operational_penalties=operational_penalties,
        construction_penalties=construction_penalties
    )
    # If successful, extract the detailed results.
    results = extract_detailed_results(network, scenario_name, base_capital_costs, base_marginal_costs)
    
    # Add metadata to the results for later analysis.
    results['sweep_variable'] = 'Economic_Baseline'
    results['primary_weight'] = 0
    
    # Append the completed scenario results to the master list.
    all_sweep_results.append(results)
except Exception as e:
    # If an error occurs, print it and continue, so one failed run doesn't stop the entire analysis.
    print(f"\n---> CRITICAL ERROR in Economic Baseline: {e}")


# --- 3. Run the Main Trade-Off Analysis Loop ---
# This section iterates through each priority, making it the primary focus of the sweep.
for sweep_variable in priorities_to_sweep:
    print(f"\n========================================================")
    print(f"--- Starting Trade-Off Analysis for: {sweep_variable.upper()} ---")
    print(f"========================================================")

    # This inner loop tests different weights for the current `sweep_variable`.
    for weight in weight_range:
        # Safety check: skip any configuration where the primary weight exceeds the total allowed sum.
        if weight > TOTAL_WEIGHT_SUM:
            print(f"--> SKIPPING: Weight {weight} for {sweep_variable} is > {TOTAL_WEIGHT_SUM}.")
            continue
            
        # --- Define Weights for the Current Scenario ---
        # The core of the trade-off analysis: the weight for the sweep_variable is set
        # explicitly, and the remaining weight is distributed evenly among all other priorities.
        current_penalty_weights = {p: 0 for p in priorities_to_sweep}
        current_penalty_weights[sweep_variable] = weight
        
        num_other_priorities = len(priorities_to_sweep) - 1
        remaining_weight = TOTAL_WEIGHT_SUM - weight
        
        # Distribute the remaining weight to maintain the constant sum.
        if num_other_priorities > 0:
            weight_for_others = remaining_weight / num_other_priorities
            for p in priorities_to_sweep:
                if p != sweep_variable:
                    current_penalty_weights[p] = weight_for_others
        
        # Create a unique name for this specific scenario run.
        scenario_name = f"Sweep_{sweep_variable}_w{weight}"
        
        # Use a try-except block to ensure that one failed scenario doesn't halt the entire loop.
        try:
            # Build and optimize the network with the calculated penalty weights for this scenario.
            network = build_and_optimize_network(
                scenario_name=scenario_name,
                penalty_weights=current_penalty_weights,
                subsampled_data=subsampled_data,
                relevant_carriers=relevant_carriers,
                base_marginal_costs=base_marginal_costs,
                base_capital_costs=base_capital_costs,
                operational_penalties=operational_penalties,
                construction_penalties=construction_penalties
            )
            # If successful, extract the detailed results.
            results = extract_detailed_results(network, scenario_name, base_capital_costs, base_marginal_costs)
            
            # Add metadata to identify which variable and weight this run corresponds to.
            results['sweep_variable'] = sweep_variable
            results['primary_weight'] = weight # The main weight for this run
            
            # Append the completed scenario results to the master list.
            all_sweep_results.append(results)
        except Exception as e:
            # If an error occurs, print it and allow the loop to continue to the next scenario.
            print(f"\n---> CRITICAL ERROR in run '{scenario_name}': {e}")

# --- 4. Consolidate All Results ---
# Aggregating all results into a single pandas DataFrame makes it easy to save, plot,
# and analyze the outcomes of the entire sweep in a structured way.
print("\n\n========================================================")
print("--- All Trade-Off Analyses Finished ---")
print("========================================================")

all_sweeps_df = pd.DataFrame(all_sweep_results)


--- Running Economic Baseline (All Penalties Zero) ---

--- Running Scenario: Economic_Baseline ---


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 8/8 [00:00<00:00, 285.41it/s]
INFO:linopy.io: Writing time: 0.24s


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-gsj9_gig has 70855 rows; 31070 cols; 139095 nonzeros
Coefficient ranges:
  Matrix [1e-03, 2e+01]
  Cost   [2e+01, 2e+08]
  Bound  [0e+00, 0e+00]
  RHS    [4e+01, 3e+04]
Presolving model
35405 rows, 31070 cols, 103645 nonzeros  0s
Dependent equations search running on 8746 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
35377 rows, 31042 cols, 103561 nonzeros  0s
Presolve : Reductions: rows 35377(-35478); columns 31042(-28); elements 103561(-35534)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3285(3.43931e+07) 0s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 31070 primals, 70855 duals
Objective: 4.55e+11
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


      17022     4.5504284446e+11 Pr: 0(0); Du: 0(2.91695e-10) 1s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-gsj9_gig
Model status        : Optimal
Simplex   iterations: 17022
Objective value     :  4.5504284446e+11
P-D objective error :  6.0358756646e-16
HiGHS run time      :          0.76
Writing the solution to /private/var/folders/h9/ylqhszz50d9dcx3w0hkvc7l40000gp/T/linopy-solve-ln81a6cq.sol
Optimization status for Economic_Baseline: ok

--- Extracting Results for: Economic_Baseline ---
   Energy balance is within tolerance (0.0000%).

--- Starting Trade-Off Analysis for: GHG ---

--- Running Scenario: Sweep_ghg_w0 ---


...

      47971     1.6844538365e+13 Pr: 0(0); Du: 0(1.32604e-08) 15s
      47971     1.6844538365e+13 Pr: 0(0); Du: 0(1.32604e-08) 15s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-nn34vlj9
Model status        : Optimal
Simplex   iterations: 47971
Objective value     :  1.6844538365e+13
P-D objective error :  5.2177520746e-16
HiGHS run time      :         14.81
Writing the solution to /private/var/folders/h9/ylqhszz50d9dcx3w0hkvc7l40000gp/T/linopy-solve-zuu1iykg.sol
Optimization status for Sweep_reliability_w50: ok

--- Extracting Results for: Sweep_reliability_w50 ---
   Energy balance is within tolerance (0.0000%).

--- Running Scenario: Sweep_reliability_w100 ---


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 8/8 [00:00<00:00, 286.62it/s]
INFO:linopy.io: Writing time: 0.21s


Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   linopy-problem-gqxwabw6 has 70855 rows; 31070 cols; 139095 nonzeros
Coefficient ranges:
  Matrix [1e-03, 2e+01]
  Cost   [2e+01, 2e+08]
  Bound  [0e+00, 0e+00]
  RHS    [4e+01, 3e+04]
Presolving model
35405 rows, 31070 cols, 103645 nonzeros  0s
Dependent equations search running on 8746 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
35377 rows, 31042 cols, 103561 nonzeros  0s
Presolve : Reductions: rows 35377(-35478); columns 31042(-28); elements 103561(-35534)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3285(3.43931e+07) 0s


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 31070 primals, 70855 duals
Objective: 1.01e+13
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance were not assigned to the network.


      20838     1.0100821579e+13 Pr: 0(0); Du: 0(5.15726e-09) 3s
Solving the original LP from the solution after postsolve
Model name          : linopy-problem-gqxwabw6
Model status        : Optimal
Simplex   iterations: 20838
Objective value     :  1.0100821579e+13
P-D objective error :  1.8369483468e-15
HiGHS run time      :          2.95
Writing the solution to /private/var/folders/h9/ylqhszz50d9dcx3w0hkvc7l40000gp/T/linopy-solve-cc_w6u01.sol
Optimization status for Sweep_reliability_w100: ok

--- Extracting Results for: Sweep_reliability_w100 ---
   Energy balance is within tolerance (0.0000%).


--- All Trade-Off Analyses Finished ---


In [9]:
# Create lists of the column names for onshore and offshore wind
onshore_cols = [f'W_On_Z{i}' for i in range(1, 10)]
offshore_cols = [f'W_Off_Z{i}' for i in range(1, 10) if f'W_Off_Z{i}' in subsampled_data.columns]
tidal_cols = [f"Z{i}_tidal" for i in [1, 3, 5, 7, 9] if f"Z{i}_tidal" in subsampled_data.columns]

  
# Calculate the average capacity factor across all zones and time
avg_onshore_cf = subsampled_data[onshore_cols].mean().mean()
avg_offshore_cf = subsampled_data[offshore_cols].mean().mean()
tidal_cf =  subsampled_data[tidal_cols].mean().mean()

print(f"Average Onshore Wind Capacity Factor: {avg_onshore_cf:.2%}")
print(f"Average Offshore Wind Capacity Factor: {avg_offshore_cf:.2%}")
print(f"Average Tidal Capacity Factor: {tidal_cf:.2%}")

Average Onshore Wind Capacity Factor: 29.49%
Average Offshore Wind Capacity Factor: 44.16%
Average Tidal Capacity Factor: 39.14%


In [13]:
#save results to csv
all_sweeps_df.to_csv("all_sweeps_results.csv", index=False)

In [9]:
all_sweeps_df.columns

Index(['Scenario', 'Optimization_Objective', 'Economic_System_Cost_GBP',
       'Installed_AC_MW', 'Installed_hydroelectric_ph_MW',
       'Installed_offwind_MW', 'Installed_onwind_MW', 'Fixed_Capacity_DSR_MW',
       'Fixed_Capacity_load_shed_MW', 'Total_Generation_MWh', 'Gen_DSR_%',
       'Gen_battery_%', 'Gen_hydroelectric_ph_%', 'Gen_load_shed_%',
       'Gen_offwind_%', 'Gen_onwind_%', 'Gen_tidal_%', 'Total_Curtailment_MWh',
       'Curtailment_('Generator', 'onwind')_MWh',
       'Curtailment_('StorageUnit', 'onwind')_MWh',
       'Curtailment_('Generator', 'offwind')_MWh',
       'Curtailment_('StorageUnit', 'offwind')_MWh',
       'Curtailment_('Generator', 'tidal')_MWh',
       'Curtailment_('StorageUnit', 'tidal')_MWh',
       'Curtailment_('Generator', 'battery')_MWh',
       'Curtailment_('StorageUnit', 'battery')_MWh',
       'Curtailment_('Generator', 'hydroelectric_ph')_MWh',
       'Curtailment_('StorageUnit', 'hydroelectric_ph')_MWh',
       'Curtailment_('Generator',